In [20]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
%autoreload

import h2o
import pandas as pd
import seaborn as sns
import utilities as utils

from h2o.automl import H2OAutoML

### Import data and format with utilities.py

In [22]:
data_list = utils.read_data()
Data = utils.DataProcessor(data_list)
df = Data.create_train_data()
df.sort_index(inplace=True)
df = df.loc['2019-09-04 08:00:00':]

df.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_w_1000hPa:ms,hour,day_of_week,quarter,month,year,day_of_year,day_of_month,building,pv_measurement
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-09-04 08:00:00,6.9,1.231,2103.100098,1166053.375,382.100006,2103.100098,0.0,278.600006,66.400002,226964.906250,...,-0.0,8,2,3,9,2019,247,4,1,570.1125
2019-09-04 08:00:00,6.8,1.226,2104.000000,1166048.250,382.100006,2104.000000,0.0,278.600006,66.400002,226962.703125,...,-0.0,8,2,3,9,2019,247,4,0,3786.2000
2019-09-04 08:00:00,6.7,1.223,2122.699951,1167217.875,382.500000,2122.699951,0.0,278.399994,66.099998,225542.296875,...,-0.0,8,2,3,9,2019,247,4,2,137.2000
2019-09-04 09:00:00,6.7,1.225,2476.500000,1559157.375,479.700012,2476.500000,0.0,278.399994,73.199997,251260.000000,...,-0.0,9,2,3,9,2019,247,4,1,542.5125
2019-09-04 09:00:00,6.7,1.220,2478.399902,1559161.250,479.700012,2478.399902,0.0,278.399994,73.199997,251256.593750,...,-0.0,9,2,3,9,2019,247,4,0,3400.7600


In [23]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 mins 17 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_stinky_7vdmhj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.796 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


## Feature Engineering with Featurewiz
Featurewiz not working on Windows desktop. Use features from featurewiz from Ubuntu.

In [24]:
FEATURES = ['elevation:m', 'sun_elevation:d', 'direct_rad_1h:J', 'sun_azimuth:d', 'effective_cloud_cover:p', 'msl_pressure:hPa', 't_1000hPa:K', 'day_of_year', 'wind_speed_u_10m:ms', 'ceiling_height_agl:m',
            'visibility:m', 'relative_humidity_1000hPa:p', 'year', 'day_of_month', 'day_of_week', 'fresh_snow_24h:cm', 'fresh_snow_6h:cm', 'fresh_snow_1h:cm', 'pv_measurement']

X_train_wiz = df[FEATURES]

## AutoML with H2O

In [25]:
data = X_train_wiz.copy()

# Convert to h2o frame
train = h2o.H2OFrame(data)
y = 'pv_measurement'
X = train.columns
X.remove(y)

print(X)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['elevation:m', 'sun_elevation:d', 'direct_rad_1h:J', 'sun_azimuth:d', 'effective_cloud_cover:p', 'msl_pressure:hPa', 't_1000hPa:K', 'day_of_year', 'wind_speed_u_10m:ms', 'ceiling_height_agl:m', 'visibility:m', 'relative_humidity_1000hPa:p', 'year', 'day_of_month', 'day_of_week', 'fresh_snow_24h:cm', 'fresh_snow_6h:cm', 'fresh_snow_1h:cm']


In [26]:
# Initialize and train the H2OAutoML model
aml = H2OAutoML(max_models=30, seed=1)
aml.train(x=X, y=y, training_frame=train)

AutoML progress: |██ (cancelled)

08:43:30.975: Empty leaderboard.
AutoML was not able to build any model within a max runtime constraint of 0 seconds, you may want to increase this value before retrying.



H2OJobCancelled: Job<$03017f00000132d4ffffffff$_8311f32d1e8562f350486c5c14f545bb> was cancelled by the user.

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

model_id                                                    rmse       mse       mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231011_214144     231.577   53627.9   78.4249  nan                          53627.9
StackedEnsemble_BestOfFamily_1_AutoML_1_20231011_214144  235.659   55535     76.933   nan                          55535
GBM_grid_1_AutoML_1_20231011_214144_model_5              236.244   55811.3   76.0494  nan                          55811.3
GBM_4_AutoML_1_20231011_214144                           237.953   56621.5   82.5077  nan                          56621.5
GBM_3_AutoML_1_20231011_214144                           242.071   58598.3   83.5382  nan                          58598.3
GBM_1_AutoML_1_20231011_214144                           246.49    60757.2   87.7614  nan                          60757.2
GBM_2_AutoML_1_20231011_214144                           247.148   61082.1   86.4555  nan                          61082.1
DRF_1_AutoML_1_202

In [ ]:
# Predict on the test data
test_pd = utils.create_test_data()
test = h2o.H2OFrame(test_pd)

preds = aml.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [ ]:
# Cast preds to pd.DataFrame and save to csv
preds_df = preds.as_data_frame()

# Set all negative values to 0
preds_df[preds_df < 0] = 0

preds_df.to_csv('h2o_automl_preds2.csv', index=True)